# Cluster and Family Label

Code for clustering proteins and nucleic acids by sequence and code for labeling protein families.

## Setup

In [ ]:
import os
import shutil

import numpy as np
import pandas as pd

## Paths and Directories

In [ ]:
# Directory of all datasets.
dataset_directory = "/home/akubaney/projects/na_mpnn/data/datasets"

# List of datasets to family label and cluster.
dataset_names = ["rcsb_cif_na", "rf2na_distillation_cis_bp", "rf2na_distillation_transfac"]

# Directory for protein family labeling.
protein_family_labeling_directory = "./protein_family_labeling"
num_jobs_for_family_labeling = 1000

# Directory for clustering.
clustering_directory = "./clustering"

## Helper Functions

In [ ]:
def read_text_file(path):
    with open(path, mode = "rt") as f:
        return f.read()

def write_text_file(path, contents):
    with open(path, mode = "wt") as f:
        f.write(contents)

def read_fasta_file(path):
    fasta_text = read_text_file(path).strip()
    fasta_entries = fasta_text[1:].split("\n>")
    id_and_sequence_pairs = []
    for fasta_entry in fasta_entries:
        header, sequence = fasta_entry.strip().split("\n")
        id = header.strip()
        id_and_sequence_pairs.append((id, sequence))
    return id_and_sequence_pairs

def write_fasta_file(path, id_and_sequence_pairs):
    fasta_entries = []
    for id, sequence in id_and_sequence_pairs:
        fasta_entries.append(f">{id}\n{sequence}")
    fasta_text = "\n".join(fasta_entries)
    write_text_file(path, fasta_text)

def read_cdhit_cluster_file(path):
    clusters_text = read_text_file(path).strip()
    cluster_entries = clusters_text[1:].split("\n>")
    clusters = dict()
    for cluster_entry in cluster_entries:
        cluster_entry_lines = cluster_entry.strip().split("\n")

        # Extract the cluster id from the header.
        cluster_header_line = cluster_entry_lines[0]
        cluster_id = int(cluster_header_line.strip().split(" ")[1])

        # Extract the cluster members.
        cluster_member_lines = cluster_entry_lines[1:]
        cluster_members = []
        for cluster_member_line in cluster_member_lines:
            member_length, member_entry = cluster_member_line.strip().split(", >")
            member_id, _ = member_entry.split("...")
            cluster_members.append(member_id)

        clusters[cluster_id] = cluster_members
    
    return clusters

In [ ]:
def get_cluster_df_from_sequences_and_clusters(fasta_path, 
                                               clusters_path,
                                               cluster_id_column_name):
    # Load the original fasta file.
    id_and_sequence_pairs = read_fasta_file(fasta_path)
    print(f"Initial number of sequences: {len(id_and_sequence_pairs)}")

    # Create a dataframe for the sequences and their cluster ids.
    cluster_data_dict = {"id": [], "sequence": []}

    for id, sequence in id_and_sequence_pairs:
        cluster_data_dict["id"].append(id)
        cluster_data_dict["sequence"].append(sequence)

    cluster_df = pd.DataFrame(cluster_data_dict)

    # Load the clusters.
    clusters = read_cdhit_cluster_file(clusters_path)
    print(f"Number of clusters: {len(clusters)}")

    # Count for sequences that failed to cluster.
    all_sequence_ids = set(cluster_df["id"])

    clustered_sequence_ids = set()
    for cluster_id in clusters:
        clustered_sequence_ids.update(clusters[cluster_id])

    unclustered_sequence_ids = all_sequence_ids.difference(clustered_sequence_ids)
    print(f"Number of unclustered sequences: {len(unclustered_sequence_ids)}")

    # Create a mapping of sequence id to cluster id.
    sequence_id_to_cluster_id = dict()
    for cluster_id in clusters:
        for sequence_id in clusters[cluster_id]:
            assert(sequence_id not in sequence_id_to_cluster_id)
            sequence_id_to_cluster_id[sequence_id] = cluster_id
    print(f"Number of sequence id to cluster id mappings: {len(sequence_id_to_cluster_id)}")

    # Add the cluster id to the dataframe.
    cluster_df[cluster_id_column_name] = cluster_df["id"].map(sequence_id_to_cluster_id)

    # Drop the sequences that failed to cluster.
    cluster_df = cluster_df.dropna(subset = [cluster_id_column_name])

    # Make the cluster id an integer.
    cluster_df[cluster_id_column_name] = cluster_df[cluster_id_column_name].astype(int)

    return cluster_df

# Family Label All Protein Sequences

## Setup

In [ ]:
# Create the directory for family labeling.
if os.path.exists(protein_family_labeling_directory):
    shutil.rmtree(protein_family_labeling_directory)
os.makedirs(protein_family_labeling_directory)

## Gather All Protein Sequnces

In [ ]:
all_protein_sequences = set()
# For every dataset.
for dataset_name in dataset_names:
    preprocessing_output = os.path.join(dataset_directory, 
                                        dataset_name, 
                                        "preprocessing_output.csv")
    preprocessing_output_df = pd.read_csv(preprocessing_output)

    # For every structure in the dataset.
    for sequences_path in preprocessing_output_df["sequences_path"]:
        sequences_df = pd.read_csv(sequences_path)

        # For every chain sequence in the structure.
        for chain_type, sequence in zip(sequences_df["chain_type"], sequences_df["sequence"]):
            # If the chain is a protein, add the sequence.
            if chain_type == "polypeptide(L)":
                all_protein_sequences.add(sequence)

print(len(all_protein_sequences))

## Save the Gathered Sequences

In [ ]:
all_protein_sequences_path = os.path.join(protein_family_labeling_directory, "all_protein_sequences.fa")
write_fasta_file(all_protein_sequences_path, enumerate(all_protein_sequences))

## Run InterScanPro and Process Results

In [ ]:
# Make the directory for the fasta splits.
fasta_splits_directory = os.path.join(protein_family_labeling_directory, "fasta_splits")
os.makedirs(fasta_splits_directory, exist_ok = True)

In [ ]:
# Split the all protein sequence fasta into smaller fastas.
all_protein_sequences_path = os.path.join(protein_family_labeling_directory, "all_protein_sequences.fa")
id_and_sequence_pairs = read_fasta_file(all_protein_sequences_path)
num_sequences_per_job = (len(all_protein_sequences) + num_jobs_for_family_labeling - 1) // num_jobs_for_family_labeling
for job_index in range(num_jobs_for_family_labeling):
    start_index = job_index * num_sequences_per_job
    end_index = min((job_index + 1) * num_sequences_per_job, len(all_protein_sequences))

    split_fasta_path = os.path.join(fasta_splits_directory, 
                                    f"all_protein_sequences_{job_index}.fa")
    write_fasta_file(split_fasta_path, id_and_sequence_pairs[start_index : end_index])

In [ ]:
# Make the directory for the family labeling outputs.
family_label_output_directory = os.path.join(protein_family_labeling_directory, "out")
os.makedirs(family_label_output_directory, exist_ok = True)

Run the following:

```
cd /home/akubaney/projects/na_mpnn/data

fasta_splits_directory="./protein_family_labeling/fasta_splits"
family_label_output_directory="./protein_family_labeling/out"
family_label_tmp_path="./protein_family_labeling/family_label_tmp.out"

rm $family_label_tmp_path

sbatch --output=$family_label_tmp_path --array=0-999 ./family_label.sh $fasta_splits_directory $family_label_output_directory
```

In [ ]:
interproscan_column_names = [
    "protein_accession", 
    "sequence_md5", 
    "sequence_length", 
    "analysis", 
    "signature_accession", 
    "signature_description", 
    "start_location", 
    "stop_location", 
    "score", 
    "status", 
    "date", 
    "interpro_accession", 
    "interpro_description", 
    "go_annotations",
    "pathway_annotations"
]

In [ ]:
# Load the protein family labels.
protein_family_label_dfs = []
for protein_family_label_csv_name in os.listdir(family_label_output_directory):
    protein_family_label_csv_path = os.path.join(family_label_output_directory, 
                                                 protein_family_label_csv_name)
    protein_family_label_df = pd.read_csv(protein_family_label_csv_path, 
                                          names = interproscan_column_names, 
                                          sep = "\t")

    protein_family_label_dfs.append(protein_family_label_df)

# Combine the results.
all_protein_family_label_df = pd.concat(protein_family_label_dfs, ignore_index = True)

# Sort by temporary id.
all_protein_family_label_df = all_protein_family_label_df.sort_values(by = "protein_accession", 
                                                                      ignore_index = True)

In [ ]:
# Load the original fasta file.
all_protein_sequences_path = os.path.join(protein_family_labeling_directory, "all_protein_sequences.fa")
id_and_sequence_pairs = read_fasta_file(all_protein_sequences_path)

# Turn the temporary id back into the sequence.
temp_id_to_sequence = dict(id_and_sequence_pairs)
all_protein_family_label_df["sequence"] = all_protein_family_label_df["protein_accession"].astype(str).map(temp_id_to_sequence)

# Drop the temporary id.
all_protein_family_label_df = all_protein_family_label_df.drop(columns = ["protein_accession"])

In [ ]:
all_protein_family_label_df

In [ ]:
# Save the results.
protein_family_label_output_path = os.path.join(protein_family_labeling_directory, "all_protein_family_labels.csv")
all_protein_family_label_df.to_csv(protein_family_label_output_path, index = False)

# Clustering

In [ ]:
# Create the directory for clustering.
if os.path.exists(clustering_directory):
    shutil.rmtree(clustering_directory)
os.makedirs(clustering_directory)

# Cluster All Protein Sequences

In [ ]:
# Create the directory for the all protein sequences clustering.
all_protein_sequences_clustering_directory = os.path.join(clustering_directory, "all_protein_sequences")
if os.path.exists(all_protein_sequences_clustering_directory):
    shutil.rmtree(all_protein_sequences_clustering_directory)
os.makedirs(all_protein_sequences_clustering_directory)

## Run CD-HIT on the Protein Sequences

```
cd /home/akubaney/projects/na_mpnn/data

input_fasta_path="./protein_family_labeling/all_protein_sequences.fa"
output_path="./clustering/all_protein_sequences/all_protein_sequences"
sequence_similarity_cutoff=0.4
length_similarity_cutoff=0.9
word_size=2

/home/akubaney/software/cd-hit-v4.8.1-2019-0228/cd-hit \
    -i $input_fasta_path \
    -o $output_path \
    -c $sequence_similarity_cutoff \
    -n $word_size \
    -d 0 \
    -M 16000 \
    -T 0 \
    -aL $length_similarity_cutoff \
    -aS $length_similarity_cutoff
```

## Compute and Save Chain Sequence Clusters

In [ ]:
# The fasta path for all protein sequences.
all_protein_sequences_path = os.path.join(protein_family_labeling_directory, "all_protein_sequences.fa")

# The output path for the cd-hit clustering.
protein_clusters_path = os.path.join(all_protein_sequences_clustering_directory, "all_protein_sequences.clstr")

# Create a dataframe that contains the protein sequences and their cluster ids.
protein_cluster_df = get_cluster_df_from_sequences_and_clusters(all_protein_sequences_path, 
                                                                protein_clusters_path,
                                                                "protein_chain_cluster_id")

In [ ]:
protein_cluster_df

In [ ]:
# Save the results.
protein_cluster_out_path = os.path.join(all_protein_sequences_clustering_directory, "all_protein_sequences_clusters.csv")
protein_cluster_df.to_csv(protein_cluster_out_path, index = False)

# Cluster All Nucleic Acid Sequences

In [ ]:
# Create the directory for the all nucleic acid sequences clustering.
all_nucleic_acid_sequences_clustering_directory = os.path.join(clustering_directory, "all_nucleic_acid_sequences")
if os.path.exists(all_nucleic_acid_sequences_clustering_directory):
    shutil.rmtree(all_nucleic_acid_sequences_clustering_directory)
os.makedirs(all_nucleic_acid_sequences_clustering_directory)

## Gather All Nucleic Acid Sequences

In [ ]:
def standardize_na_sequence(sequence):
    """
    Given a nucleic acid sequence string, map U->T, and any residue that is not
    A,C,G,T to X.
    """
    nucleic_acid_mapping = {
        "A": "A",
        "C": "C",
        "G": "G",
        "T": "T",
        "U": "T"
    }

    sequence = "".join(list(map(lambda c: nucleic_acid_mapping.get(c, "X"), list(sequence))))
    return sequence

In [ ]:
all_nucleic_acid_sequences = set()
all_nucleic_acid_standard_sequences = set()
# For every dataset.
for dataset_name in dataset_names:
    preprocessing_output = os.path.join(dataset_directory, 
                                        dataset_name, 
                                        "preprocessing_output.csv")
    preprocessing_output_df = pd.read_csv(preprocessing_output)

    # For every structure in the dataset.
    for sequences_path in preprocessing_output_df["sequences_path"]:
        sequences_df = pd.read_csv(sequences_path)

        # For every chain sequence in the structure.
        for chain_type, sequence in zip(sequences_df["chain_type"], sequences_df["sequence"]):
            # If the chain is a nucleic acid, add the sequence.
            if chain_type in ["polydeoxyribonucleotide/polyribonucleotide hybrid", 
                              "polydeoxyribonucleotide", 
                              "polyribonucleotide"]:
                all_nucleic_acid_sequences.add(sequence)
                all_nucleic_acid_standard_sequences.add(standardize_na_sequence(sequence))

print(len(all_nucleic_acid_sequences))
print(len(all_nucleic_acid_standard_sequences))

In [ ]:
all_nucleic_acid_sequences_path = os.path.join(all_nucleic_acid_sequences_clustering_directory, 
                                               "all_nucleic_acid_sequences.fa")
write_fasta_file(all_nucleic_acid_sequences_path, enumerate(all_nucleic_acid_sequences))

all_nucleic_acid_standard_sequences_path = os.path.join(all_nucleic_acid_sequences_clustering_directory, 
                                                        "all_nucleic_acid_standard_sequences.fa")
write_fasta_file(all_nucleic_acid_standard_sequences_path, enumerate(all_nucleic_acid_standard_sequences))

## Run CD-HIT on the Nucleic Acid Sequences

```
cd /home/akubaney/projects/na_mpnn/data

input_fasta_path="./clustering/all_nucleic_acid_sequences/all_nucleic_acid_standard_sequences.fa"
output_path="./clustering/all_nucleic_acid_sequences/all_nucleic_acid_standard_sequences"
sequence_similarity_cutoff=0.8
length_similarity_cutoff=0.9
word_size=4
length_limit=4

/home/akubaney/software/cd-hit-v4.8.1-2019-0228/cd-hit-est \
    -i $input_fasta_path \
    -o $output_path \
    -c $sequence_similarity_cutoff \
    -n $word_size \
    -d 0 \
    -M 16000 \
    -T 0 \
    -l $length_limit \
    -aL $length_similarity_cutoff \
    -aS $length_similarity_cutoff
```

## Compute and Save Chain Sequence Clusters

In [ ]:
# The fasta path for all nucleic acid standard sequences.
all_nucleic_acid_standard_sequences_path = os.path.join(all_nucleic_acid_sequences_clustering_directory, 
                                                        "all_nucleic_acid_standard_sequences.fa")

# The output path for the cd-hit clustering.
nucleic_acid_standard_clusters_path = os.path.join(all_nucleic_acid_sequences_clustering_directory, 
                                                   "all_nucleic_acid_standard_sequences.clstr")

# Create a dataframe that contains the nucleic acid standard sequences and their cluster ids.
nucleic_acid_standard_cluster_df = \
    get_cluster_df_from_sequences_and_clusters(all_nucleic_acid_standard_sequences_path, 
                                               nucleic_acid_standard_clusters_path,
                                               "nucleic_acid_standard_chain_cluster_id")
standard_sequence_to_cluster_id = dict(zip(nucleic_acid_standard_cluster_df["sequence"], 
                                           nucleic_acid_standard_cluster_df["nucleic_acid_standard_chain_cluster_id"]))

# Create a dataframe that contains the nucleic acid sequences and their standard cluster ids.
all_nucleic_acid_sequences_path = os.path.join(all_nucleic_acid_sequences_clustering_directory,
                                               "all_nucleic_acid_sequences.fa")
id_and_sequence_pairs = read_fasta_file(all_nucleic_acid_sequences_path)

nucleic_acid_cluster_data_dict = {"id": [], "sequence": [], "nucleic_acid_chain_cluster_id": []}
for id, sequence in id_and_sequence_pairs:
    standard_sequence = standardize_na_sequence(sequence)
    # Since non-clustered sequences were dropped, we need to check this here.
    if standard_sequence in standard_sequence_to_cluster_id:
        cluster_id = standard_sequence_to_cluster_id[standard_sequence]
        nucleic_acid_cluster_data_dict["id"].append(id)
        nucleic_acid_cluster_data_dict["sequence"].append(sequence)
        nucleic_acid_cluster_data_dict["nucleic_acid_chain_cluster_id"].append(cluster_id)

nucleic_acid_cluster_df = pd.DataFrame(nucleic_acid_cluster_data_dict)

In [ ]:
nucleic_acid_cluster_df

In [ ]:
# Save the results.
nucleic_acid_cluster_out_path = os.path.join(all_nucleic_acid_sequences_clustering_directory, 
                                             "all_nucleic_acid_sequences_clusters.csv")
nucleic_acid_cluster_df.to_csv(nucleic_acid_cluster_out_path, index = False)